In [4]:
"""
Traveling Salesmen (Minimize distance traveled through sources and destinations)
"""

# Import Gurobi
from gurobipy import *


In [5]:
# Function to minimize travel measure
def getMinRoute(POS_NAME, POS, OBJ_MAT, objMeasureName, minHours):
    # Create model for optimization
    m = Model('Traveling Salesman')
    
    ## Create Empty Sets and Paremters to append in following block 
    
    ### Nodes
    Nodes = tuplelist([])
    
    ### Arcs
    Arcs = tuplelist([])
    
    ### Distance
    objCost = {}
    
    ### Actual Cost
    #supportCost = {}
    
    ### Read distance and position lists to create set of node (Nodes), arcs (Arcs), and
    ### Distance between nodes (objCost)
    for i, pos_i in enumerate(POS):
        Nodes.append(i)
        for j, pos_j in enumerate(POS):
            if j!= i:
                Arcs.append((i, j))
                objCost[i, j] = OBJ_MAT[i][j]
                #supportCost[i, j]   = SUPPORT_MAT[i][j]
                
    ### COunt of nodes
    numNodes = len(Nodes)
    
    # OPTIMIZATION ================================================================
    
    ## Create the model
    m = Model('Traveling Salesman')
    
    ## Create variables and the coefficients of the objective function
    isVisited    = m.addVars(Arcs,  obj = objCost, name = 'isVisited', vtype = GRB.BINARY)
    m.update()
    orderOfVisit = m.addVars(Nodes, obj = 0, name = 'orderOfVisit')
    m.update()
    
    ## Constraints ---------------------------------------------------------------
    
    ### Only can depart from a single node
    m.addConstrs(
        (isVisited.sum('*', j) == 1 for j in Nodes),
        'departureNode')
    m.update()
    
    ### Only can arrive at a single node
    m.addConstrs(
        (isVisited.sum(i, '*') == 1 for i in Nodes),
        'arrivalNode')
    m.update()
    
    ### Time labels?
    m.addConstrs(
        (numNodes*(1 - isVisited[i, j]) >= orderOfVisit[i]-orderOfVisit[j]+1 for (i,j) in Arcs if (j!=0)),
        'timeLabels')
    m.update()
    
    ### Total number of Hours traveled must be less than the minHours that you define
    #m.addConstr(sum(supportCost[node] * isVisited[node] for node in isVisited) <= minHours)
    
    ## Optimize the Minimum Distance Traveled
    m.modelSense = GRB.MINIMIZE
    m.setParam('OutputFlag', 0)
    m.update()
    m.optimize()
    
    ## Get the total cost
    #totalSupportCost = sum(supportCost[node] * isVisited[node].x for node in isVisited) 
    
    # Print the solution
    if m.status == GRB.Status.OPTIMAL:
        print('\n======== Optimized ' + objMeasureName + ' ========')
        solution_OF = m.objVal
        solution_x  = m.getAttr('x', isVisited) 
        solution_u  = m.getAttr('x', orderOfVisit)
        print('Total Optimized ' + objMeasureName + ': \t%g' % solution_OF)
        #print('Total Associated '  + suportMeasureName + ': \t%g' % totalSupportCost)
        print('\nOptimal Path:')
        for i, j in Arcs:
            if solution_x[i,j] > 0:
                print('%s\t->\t%s' % (POS_NAME[i], POS_NAME[j]))

In [6]:
## Name of the positions
POS_NAME = ['Albany, New York', 'Annapolis, Maryland', 'Atlanta, Georgia', 'Augusta, Maine', 'Austin, Texas', 'Baton Rouge, Louisiana', 'Bismarck, North Dakota', 'Boise, Idaho', 'Boston, Massachusetts', 'Carson City, Nevada', 'Charleston, West Virginia', 'Cheyenne, Wyoming', 'Columbia, South Carolina', 'Columbus, Ohio', 'Concord, New Hampshire', 'Denver, Colorado', 'Des Moines, Iowa', 'Dover, Delaware', 'Frankfort, Kentucky', 'Harrisburg, Pennsylvania', 'Hartford, Connecticut', 'Helena, Montana', 'Honolulu, Hawaii', 'Indianapolis, Indiana', 'Jackson, Mississippi', 'Jefferson City, Missouri', 'Juneau, Alaska', 'Lansing, Michigan', 'Lincoln, Nebraska', 'Little Rock, Arkansas', 'Madison, Wisconsin', 'Montgomery, Alabama', 'Montpelier, Vermont', 'Nashville, Tennessee', 'Oklahoma City, Oklahoma', 'Olympia, Washington', 'Phoenix, Arizona', 'Pierre, South Dakota', 'Providence, Rhode Island', 'Raleigh, North Carolina', 'Richmond, Virginia', 'Sacramento, California', 'Saint Paul, Minnesota', 'Salem, Oregon', 'Salt Lake City, Utah', 'Santa Fe, New Mexico', 'Springfield, Illinois', 'Tallahassee, Florida', 'Topeka, Kansas', 'Trenton, New Jersey']

# Positions
POS = [[-5097.97, 2947.61], 
        [-5285.9, 2692.86], 
        [-5830.98, 2332.67], 
        [-4820.48, 3062.56], 
        [-6754.1, 2091.3], 
        [-6297.39, 2104.52], 
        [-6963.39, 3372.79], 
        [-8031.52, 3013.52], 
        [-4907.69, 2918.27], 
        [-8274.47, 2705.85], 
        [-5640.51, 2649.78], 
        [-7241.37, 2842.98], 
        [-5599.17, 2349.25], 
        [-5734.98, 2761.22], 
        [-4943.73, 2986.32], 
        [-7253.95, 2745.8], 
        [-6468.8, 2873.75], 
        [-5218.57, 2705.92], 
        [-5863.67, 2639.27], 
        [-5311.77, 2782.47], 
        [-5021.67, 2885.92], 
        [-7740.58, 3219.57], 
        [-10905.11, 1472.36], 
        [-5952.43, 2749.38], 
        [-6232.91, 2233.17], 
        [-6369.88, 2665.22], 
        [-9287.82, 4028.41], 
        [-5841.81, 2952.7], 
        [-6679.85, 2819.78], 
        [-6379.68, 2400.11], 
        [-6176.08, 2976.28], 
        [-5961.51, 2236.04], 
        [-5014.41, 3058.62], 
        [-5996.4, 2498.84], 
        [-6739.25, 2451.67], 
        [-8491.38, 3250.43], 
        [-7743.82, 2311.14], 
        [-6932.81, 3065.63], 
        [-4934.96, 2889.83], 
        [-5433.54, 2471.62], 
        [-5352.15, 2593.85], 
        [-8392.98, 2664.03], 
        [-6432.39, 3105.85], 
        [-8500.78, 3104.54], 
        [-7731.3, 2815.97], 
        [-7321.69, 2464.45], 
        [-6194.45, 2748.85], 
        [-5822.88, 2104.09], 
        [-6611.76, 2697.49], 
        [-5165.33, 2779.15]]



# Distances between all positions
DIST = [[0, 317, 957, 300, 1864, 1466, 1913, 2934, 193, 3186, 619, 2146, 781, 664, 159, 2165, 1373, 270, 825, 270, 98, 2657, 5992, 877, 1341, 1303, 4327, 744, 1587, 1394, 1078, 1119, 139, 1004, 1715, 3407, 2721, 1839, 173, 582, 436, 3307, 1344, 3406, 2637, 2276, 1114, 1112, 1534, 181], 
        [317, 0, 653, 594, 1587, 1170, 1810, 2764, 440, 2989, 357, 1961, 465, 454, 451, 1969, 1197, 69, 580, 93, 327, 2511, 5750, 669, 1053, 1084, 4219, 614, 1400, 1132, 934, 816, 456, 737, 1473, 3254, 2487, 1689, 402, 266, 119, 3107, 1219, 3241, 2448, 2049, 910, 797, 1326, 148], 
        [957, 653, 0, 1247, 954, 519, 1538, 2303, 1093, 2472, 370, 1500, 232, 439, 1102, 1482, 836, 717, 308, 687, 980, 2106, 5147, 434, 414, 633, 3850, 620, 979, 553, 730, 162, 1093, 234, 916, 2814, 1913, 1323, 1055, 421, 545, 2583, 980, 2779, 1961, 1497, 553, 229, 862, 802], 
        [300, 594, 1247, 0, 2164, 1760, 2165, 3211, 169, 3472, 918, 2431, 1056, 963, 145, 2454, 1659, 534, 1126, 566, 268, 2924, 6289, 1174, 1638, 1600, 4571, 1027, 1875, 1694, 1358, 1409, 194, 1304, 2014, 3676, 3018, 2112, 207, 852, 709, 3595, 1612, 3681, 2921, 2572, 1409, 1387, 1828, 446], 
        [1864, 1587, 954, 2164, 0, 457, 1298, 1576, 2023, 1640, 1246, 896, 1183, 1220, 2020, 824, 833, 1654, 1046, 1599, 1906, 1499, 4197, 1037, 540, 691, 3189, 1255, 732, 485, 1057, 806, 1991, 860, 361, 2088, 1014, 991, 1987, 1374, 1489, 1736, 1064, 2019, 1217, 679, 863, 931, 623, 1731], 
        [1466, 1170, 519, 1760, 457, 0, 1433, 1958, 1610, 2067, 854, 1198, 740, 865, 1616, 1152, 788, 1235, 689, 1196, 1496, 1824, 4651, 731, 144, 565, 3556, 963, 811, 307, 880, 361, 1599, 496, 562, 2475, 1461, 1152, 1573, 939, 1064, 2169, 1010, 2420, 1601, 1086, 652, 475, 671, 1318], 
        [1913, 1810, 1538, 2165, 1298, 1433, 0, 1127, 2105, 1471, 1508, 598, 1706, 1372, 2056, 691, 703, 1868, 1322, 1754, 2002, 792, 4376, 1188, 1354, 924, 2415, 1198, 621, 1134, 882, 1515, 1974, 1303, 948, 1533, 1318, 309, 2085, 1776, 1790, 1596, 594, 1561, 949, 976, 990, 1706, 761, 1894], 
        [2934, 2764, 2303, 3211, 1576, 1958, 1127, 0, 3125, 392, 2419, 808, 2521, 2310, 3088, 822, 1569, 2830, 2200, 2730, 3013, 357, 3261, 2096, 1961, 1698, 1615, 2191, 1365, 1762, 1856, 2211, 3017, 2099, 1409, 517, 759, 1100, 3099, 2654, 2712, 503, 1602, 478, 359, 897, 1856, 2389, 1455, 2876], 
        [193, 440, 1093, 169, 2023, 1610, 2105, 3125, 0, 3373, 780, 2335, 895, 842, 77, 2353, 1562, 376, 996, 426, 118, 2849, 6169, 1058, 1492, 1484, 4519, 935, 1775, 1561, 1270, 1255, 176, 1167, 1890, 3599, 2900, 2030, 39, 690, 550, 3495, 1536, 3598, 2825, 2456, 1298, 1225, 1718, 293], 
        [3186, 2989, 2472, 3472, 1640, 2067, 1471, 392, 3373, 0, 2635, 1042, 2699, 2540, 3343, 1021, 1813, 3056, 2412, 2964, 3258, 741, 2905, 2322, 2096, 1905, 1666, 2445, 1599, 1919, 2116, 2360, 3279, 2287, 1556, 586, 661, 1389, 3345, 2851, 2924, 126, 1885, 458, 554, 983, 2080, 2524, 1663, 3110], 
        [619, 357, 370, 918, 1246, 854, 1508, 2419, 780, 2635, 0, 1612, 303, 146, 774, 1616, 858, 426, 223, 355, 662, 2176, 5395, 327, 724, 730, 3899, 364, 1053, 780, 627, 524, 748, 387, 1116, 2913, 2130, 1358, 745, 273, 294, 2753, 914, 2896, 2097, 1691, 563, 575, 972, 492], 
        [2146, 1961, 1500, 2431, 896, 1198, 598, 808, 2335, 1042, 1612, 0, 1715, 1509, 2302, 98, 773, 2027, 1393, 1931, 2220, 625, 3912, 1292, 1178, 889, 2365, 1404, 562, 969, 1074, 1416, 2237, 1292, 637, 1315, 732, 381, 2307, 1846, 1906, 1165, 851, 1286, 491, 387, 1051, 1599, 646, 2077], 
        [781, 465, 232, 1056, 1183, 740, 1706, 2521, 895, 2699, 303, 1715, 0, 434, 914, 1702, 1016, 522, 393, 520, 788, 2312, 5378, 534, 644, 833, 4053, 650, 1179, 782, 852, 380, 919, 424, 1145, 3029, 2145, 1514, 856, 206, 348, 2811, 1125, 2998, 2183, 1726, 717, 332, 1071, 611], 
        [664, 454, 439, 963, 1220, 865, 1372, 2310, 842, 2540, 146, 1509, 434, 0, 823, 1519, 742, 519, 177, 424, 724, 2057, 5328, 218, 726, 642, 3772, 219, 947, 739, 491, 572, 780, 370, 1051, 2799, 2059, 1236, 810, 418, 418, 2660, 778, 2787, 1997, 1614, 460, 663, 879, 570], 
        [159, 451, 1102, 145, 2020, 1616, 2056, 3088, 77, 3343, 774, 2302, 914, 823, 0, 2323, 1529, 393, 983, 421, 127, 2807, 6151, 1036, 1493, 1462, 4467, 899, 1744, 1551, 1232, 1264, 101, 1160, 1873, 3557, 2880, 1991, 97, 711, 566, 3464, 1493, 3559, 2793, 2435, 1273, 1245, 1693, 303], 
        [2165, 1969, 1482, 2454, 824, 1152, 691, 822, 2353, 1021, 1616, 98, 1702, 1519, 2323, 0, 796, 2036, 1394, 1943, 2237, 679, 3867, 1302, 1143, 888, 2405, 1427, 579, 940, 1102, 1389, 2261, 1282, 593, 1336, 655, 453, 2323, 1841, 1908, 1142, 897, 1297, 482, 289, 1060, 1568, 644, 2089], 
        [1373, 1197, 836, 1659, 833, 788, 703, 1569, 1562, 1813, 858, 773, 1016, 742, 1529, 796, 0, 1261, 649, 1161, 1447, 1318, 4652, 531, 683, 231, 3046, 632, 218, 482, 310, 815, 1466, 603, 501, 2057, 1394, 502, 1534, 1111, 1151, 1936, 235, 2045, 1264, 946, 301, 1005, 227, 1307], 
        [270, 69, 717, 534, 1654, 1235, 1868, 2830, 376, 3056, 426, 2027, 522, 519, 393, 2036, 1261, 0, 649, 121, 267, 2574, 5819, 735, 1119, 1152, 4279, 670, 1466, 1201, 995, 879, 408, 805, 1542, 3318, 2556, 1752, 338, 318, 174, 3175, 1278, 3306, 2515, 2117, 977, 853, 1393, 91], 
        [825, 580, 308, 1126, 1046, 689, 1322, 2200, 996, 2412, 223, 1393, 393, 177, 983, 1394, 649, 649, 0, 570, 877, 1965, 5175, 141, 549, 507, 3695, 314, 836, 569, 460, 415, 947, 193, 895, 2698, 1909, 1151, 962, 462, 514, 2529, 736, 2678, 1876, 1468, 348, 537, 750, 712], 
        [270, 93, 687, 566, 1599, 1196, 1754, 2730, 426, 2964, 355, 1931, 520, 424, 421, 1943, 1161, 121, 570, 0, 308, 2468, 5745, 642, 1072, 1065, 4167, 557, 1369, 1134, 886, 849, 406, 741, 1465, 3214, 2477, 1646, 392, 334, 193, 3083, 1166, 3205, 2420, 2035, 883, 849, 1303, 146], 
        [98, 327, 980, 268, 1906, 1496, 2002, 3013, 118, 3258, 662, 2220, 788, 724, 127, 2237, 1447, 267, 877, 308, 0, 2739, 6051, 941, 1376, 1366, 4416, 823, 1659, 1442, 1158, 1143, 173, 1049, 1772, 3489, 2782, 1920, 87, 584, 441, 3379, 1428, 3486, 2711, 2338, 1181, 1119, 1601, 179], 
        [2657, 2511, 2106, 2924, 1499, 1824, 792, 357, 2849, 741, 2176, 625, 2312, 2057, 2807, 679, 1318, 2574, 1965, 2468, 2739, 0, 3615, 1849, 1802, 1479, 1746, 1917, 1134, 1589, 1583, 2033, 2731, 1887, 1262, 751, 908, 822, 2825, 2425, 2469, 857, 1313, 769, 404, 864, 1616, 2219, 1244, 2613], 
        [5992, 5750, 5147, 6289, 4197, 4651, 4376, 3261, 6169, 2905, 5395, 3912, 5378, 5328, 6151, 3867, 4652, 5819, 5175, 5745, 6051, 3615, 0, 5115, 4734, 4689, 3025, 5275, 4435, 4620, 4962, 5002, 6101, 5015, 4279, 2998, 3271, 4280, 6136, 5562, 5665, 2780, 4762, 2906, 3447, 3718, 4881, 5121, 4465, 5887], 
        [877, 669, 434, 1174, 1037, 731, 1188, 2096, 1058, 2322, 327, 1292, 534, 218, 1036, 1302, 531, 735, 141, 642, 941, 1849, 5115, 0, 587, 426, 3572, 231, 731, 552, 319, 513, 988, 254, 841, 2588, 1844, 1030, 1027, 589, 620, 2442, 598, 2573, 1780, 1399, 242, 658, 661, 788], 
        [1341, 1053, 414, 1638, 540, 144, 1354, 1961, 1492, 2096, 724, 1178, 644, 726, 1493, 1143, 683, 1119, 549, 1072, 1376, 1802, 4734, 587, 0, 453, 3543, 819, 737, 222, 745, 271, 1472, 356, 551, 2477, 1513, 1088, 1455, 834, 952, 2203, 895, 2430, 1608, 1113, 517, 430, 599, 1199], 
        [1303, 1084, 633, 1600, 691, 565, 924, 1698, 1484, 1905, 730, 889, 833, 642, 1462, 888, 231, 1152, 507, 1065, 1366, 1479, 4689, 426, 453, 0, 3221, 601, 346, 265, 366, 592, 1411, 409, 427, 2201, 1419, 691, 1452, 956, 1020, 2023, 445, 2176, 1370, 973, 194, 784, 244, 1210], 
        [4327, 4219, 3850, 4571, 3189, 3556, 2415, 1615, 4519, 1666, 3899, 2365, 4053, 3772, 4467, 2405, 3046, 4279, 3695, 4167, 4416, 1746, 3025, 3572, 3543, 3221, 0, 3610, 2874, 3333, 3285, 3778, 4382, 3629, 2997, 1113, 2309, 2544, 4499, 4157, 4189, 1632, 3001, 1214, 1973, 2512, 3348, 3963, 2989, 4308], 
        [744, 614, 620, 1027, 1255, 963, 1198, 2191, 935, 2445, 364, 1404, 650, 219, 899, 1427, 632, 670, 314, 557, 823, 1917, 5275, 231, 819, 601, 3610, 0, 849, 771, 335, 727, 834, 479, 1028, 2666, 2007, 1097, 909, 631, 607, 2567, 610, 2663, 1894, 1558, 407, 849, 811, 698], 
        [1587, 1400, 979, 1875, 732, 811, 621, 1365, 1775, 1599, 1053, 562, 1179, 947, 1744, 579, 218, 1466, 836, 1369, 1659, 1134, 4435, 731, 737, 346, 2874, 849, 0, 516, 528, 926, 1682, 755, 373, 1862, 1179, 353, 1746, 1294, 1347, 1720, 378, 1843, 1051, 734, 491, 1117, 140, 1515], 
        [1394, 1132, 553, 1694, 485, 307, 1134, 1762, 1561, 1919, 780, 969, 782, 739, 1551, 940, 482, 1201, 569, 1134, 1442, 1589, 4620, 552, 222, 265, 3333, 771, 516, 0, 611, 449, 1516, 396, 363, 2276, 1367, 865, 1525, 949, 1046, 2031, 708, 2235, 1414, 944, 395, 631, 377, 1272], 
        [1078, 934, 730, 1358, 1057, 880, 882, 1856, 1270, 2116, 627, 1074, 852, 491, 1232, 1102, 310, 995, 460, 886, 1158, 1583, 4962, 319, 745, 366, 3285, 335, 528, 611, 0, 771, 1165, 510, 770, 2331, 1703, 762, 1244, 898, 908, 2239, 287, 2328, 1563, 1255, 228, 941, 517, 1030], 
        [1119, 816, 162, 1409, 806, 361, 1515, 2211, 1255, 2360, 524, 1416, 380, 572, 1264, 1389, 815, 879, 415, 849, 1143, 2033, 5002, 513, 271, 592, 3778, 727, 926, 449, 771, 0, 1254, 265, 807, 2726, 1784, 1277, 1217, 578, 707, 2469, 989, 2684, 1862, 1379, 563, 191, 797, 964], 
        [139, 456, 1093, 194, 1991, 1599, 1974, 3017, 176, 3279, 748, 2237, 919, 780, 101, 2261, 1466, 408, 947, 406, 173, 2731, 6101, 988, 1472, 1411, 4382, 834, 1682, 1516, 1165, 1254, 0, 1130, 1829, 3482, 2830, 1918, 187, 721, 575, 3402, 1419, 3487, 2728, 2383, 1220, 1251, 1638, 318], 
        [1004, 737, 234, 1304, 860, 496, 1303, 2099, 1167, 2287, 387, 1292, 424, 370, 1160, 1282, 603, 805, 193, 741, 1049, 1887, 5015, 254, 356, 409, 3629, 479, 755, 396, 510, 265, 1130, 0, 744, 2606, 1757, 1095, 1131, 564, 651, 2402, 747, 2577, 1764, 1326, 319, 431, 647, 877], 
        [1715, 1473, 916, 2014, 361, 562, 948, 1409, 1890, 1556, 1116, 637, 1145, 1051, 1873, 593, 501, 1542, 895, 1465, 1772, 1262, 4279, 841, 551, 427, 2997, 1028, 373, 363, 770, 807, 1829, 744, 0, 1926, 1014, 644, 1857, 1306, 1394, 1667, 723, 1879, 1057, 583, 621, 980, 277, 1608], 
        [3407, 3254, 2814, 3676, 2088, 2475, 1533, 517, 3599, 586, 2913, 1315, 3029, 2799, 3557, 1336, 2057, 3318, 2698, 3214, 3489, 751, 2998, 2588, 2477, 2201, 1113, 2666, 1862, 2276, 2331, 2726, 3482, 2606, 1926, 0, 1200, 1569, 3575, 3155, 3207, 595, 2064, 146, 875, 1409, 2351, 2904, 1959, 3359], 
        [2721, 2487, 1913, 3018, 1014, 1461, 1318, 759, 2900, 661, 2130, 732, 2145, 2059, 2880, 655, 1394, 2556, 1909, 2477, 2782, 908, 3271, 1844, 1513, 1419, 2309, 2007, 1179, 1367, 1703, 1784, 2830, 1757, 1014, 1200, 0, 1108, 2868, 2316, 2408, 739, 1533, 1097, 505, 449, 1610, 1932, 1196, 2621], 
        [1839, 1689, 1323, 2112, 991, 1152, 309, 1100, 2030, 1389, 1358, 381, 1514, 1236, 1991, 453, 502, 1752, 1151, 1646, 1920, 822, 4280, 1030, 1088, 691, 2544, 1097, 353, 865, 762, 1277, 1918, 1095, 644, 1569, 1108, 0, 2006, 1613, 1650, 1514, 502, 1568, 837, 716, 803, 1469, 488, 1791], 
        [173, 402, 1055, 207, 1987, 1573, 2085, 3099, 39, 3345, 745, 2307, 856, 810, 97, 2323, 1534, 338, 962, 392, 87, 2825, 6136, 1027, 1455, 1452, 4499, 909, 1746, 1525, 1244, 1217, 187, 1131, 1857, 3575, 2868, 2006, 0, 651, 512, 3465, 1513, 3572, 2797, 2424, 1267, 1186, 1688, 256], 
        [582, 266, 421, 852, 1374, 939, 1776, 2654, 690, 2851, 273, 1846, 206, 418, 711, 1841, 1111, 318, 462, 334, 584, 2425, 5562, 589, 834, 956, 4157, 631, 1294, 949, 898, 578, 721, 564, 1306, 3155, 2316, 1613, 651, 0, 147, 2966, 1183, 3132, 2323, 1888, 810, 535, 1200, 408], 
        [436, 119, 545, 709, 1489, 1064, 1790, 2712, 550, 2924, 294, 1906, 348, 418, 566, 1908, 1151, 174, 514, 193, 441, 2469, 5665, 620, 952, 1020, 4189, 607, 1347, 1046, 908, 707, 575, 651, 1394, 3207, 2408, 1650, 512, 147, 0, 3042, 1195, 3190, 2389, 1974, 856, 679, 1264, 263], 
        [3307, 3107, 2583, 3595, 1736, 2169, 1596, 503, 3495, 126, 2753, 1165, 2811, 2660, 3464, 1142, 1936, 3175, 2529, 3083, 3379, 857, 2780, 2442, 2203, 2023, 1632, 2567, 1720, 2031, 2239, 2469, 3402, 2402, 1667, 595, 739, 1514, 3465, 2966, 3042, 0, 2010, 454, 679, 1090, 2200, 2630, 1782, 3230], 
        [1344, 1219, 980, 1612, 1064, 1010, 594, 1602, 1536, 1885, 914, 851, 1125, 778, 1493, 897, 235, 1278, 736, 1166, 1428, 1313, 4762, 598, 895, 445, 3001, 610, 378, 708, 287, 989, 1419, 747, 723, 2064, 1533, 502, 1513, 1183, 1195, 2010, 0, 2068, 1331, 1096, 429, 1173, 446, 1309], 
        [3406, 3241, 2779, 3681, 2019, 2420, 1561, 478, 3598, 458, 2896, 1286, 2998, 2787, 3559, 1297, 2045, 3306, 2678, 3205, 3486, 769, 2906, 2573, 2430, 2176, 1214, 2663, 1843, 2235, 2328, 2684, 3487, 2577, 1879, 146, 1097, 1568, 3572, 3132, 3190, 454, 2068, 0, 822, 1342, 2334, 2859, 1932, 3351], 
        [2637, 2448, 1961, 2921, 1217, 1601, 949, 359, 2825, 554, 2097, 491, 2183, 1997, 2793, 482, 1264, 2515, 1876, 2420, 2711, 404, 3447, 1780, 1608, 1370, 1973, 1894, 1051, 1414, 1563, 1862, 2728, 1764, 1057, 875, 505, 837, 2797, 2323, 2389, 679, 1331, 822, 0, 540, 1538, 2037, 1126, 2566], 
        [2276, 2049, 1497, 2572, 679, 1086, 976, 897, 2456, 983, 1691, 387, 1726, 1614, 2435, 289, 946, 2117, 1468, 2035, 2338, 864, 3718, 1399, 1113, 973, 2512, 1558, 734, 944, 1255, 1379, 2383, 1326, 583, 1409, 449, 716, 2424, 1888, 1974, 1090, 1096, 1342, 540, 0, 1163, 1542, 747, 2179], 
        [1114, 910, 553, 1409, 863, 652, 990, 1856, 1298, 2080, 563, 1051, 717, 460, 1273, 1060, 301, 977, 348, 883, 1181, 1616, 4881, 242, 517, 194, 3348, 407, 491, 395, 228, 563, 1220, 319, 621, 2351, 1610, 803, 1267, 810, 856, 2200, 429, 2334, 1538, 1163, 0, 744, 420, 1030], 
        [1112, 797, 229, 1387, 931, 475, 1706, 2389, 1225, 2524, 575, 1599, 332, 663, 1245, 1568, 1005, 853, 537, 849, 1119, 2219, 5121, 658, 430, 784, 3963, 849, 1117, 631, 941, 191, 1251, 431, 980, 2904, 1932, 1469, 1186, 535, 679, 2630, 1173, 2859, 2037, 1542, 744, 0, 987, 942], 
        [1534, 1326, 862, 1828, 623, 671, 761, 1455, 1718, 1663, 972, 646, 1071, 879, 1693, 644, 227, 1393, 750, 1303, 1601, 1244, 4465, 661, 599, 244, 2989, 811, 140, 377, 517, 797, 1638, 647, 277, 1959, 1196, 488, 1688, 1200, 1264, 1782, 446, 1932, 1126, 747, 420, 987, 0, 1449], 
        [181, 148, 802, 446, 1731, 1318, 1894, 2876, 293, 3110, 492, 2077, 611, 570, 303, 2089, 1307, 91, 712, 146, 179, 2613, 5887, 788, 1199, 1210, 4308, 698, 1515, 1272, 1030, 964, 318, 877, 1608, 3359, 2621, 1791, 256, 408, 263, 3230, 1309, 3351, 2566, 2179, 1030, 942, 1449, 0]]



In [7]:
# Call function to minimize distance
getMinRoute(POS_NAME, POS, 
            OBJ_MAT=DIST, objMeasureName='Raw Distance between Cities - "As the crow flies"', 
            minHours=GRB.INFINITY)


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license